### This notebook parses the raw text data and extracts valid Doodle polls from it

In [1]:
lines = open('../Data/raw_data.txt').read().split('\n\n')  # 'hidden_polls.txt'/ 'raw_data.txt' / ifiexport2-merged.sample
all_lines = [line.splitlines() for line in lines]

In [2]:
import numpy as np
import pandas as pd

temp_list = []
for poll_idx, line in enumerate(all_lines):
    need_parse = True
    temp_dict = dict()
    temp_dict['votes'] = []
    temp_dict['metadata'] = {}
    for idx, txt in enumerate(line):
        if need_parse:
            if txt:
                id_, opt1, opt2 = txt.split(' ')
                temp_dict['metadata'].update({'id': id_, 'opt1': opt1, 'opt2': opt2})
                need_parse = False
            continue
        try:
            label, value = txt.split(': ')
            temp_dict['metadata'].update({label: value})
        except ValueError:
            if txt.isnumeric():
                temp_dict['votes'].append([vote for vote in txt])
    if temp_dict['votes']:
        temp_dict['votes'] = pd.DataFrame(temp_dict['votes'], columns=temp_dict['metadata']['Options'].split(' '))
        temp_list.append(temp_dict)
len(temp_list)

676

In [3]:
# Extract only hidden polls with more than 2 responders
good_inds = np.where([((temp_list[i]['metadata']['opt1'] == 'hidden') and 
                       (temp_list[i]['votes'].shape[0] > 2))
                        for i in range(len(temp_list))])[0]

In [4]:
# temp_list_filt = [temp_list[i] for i in good_inds]
import pickle
with open('Data/parsed_data', 'wb') as f:
    pickle.dump(temp_list_filt, f, protocol=pickle.HIGHEST_PROTOCOL)